<font color='steelblue'>

<span style="font-family:verdana; font-size:1.6em;">
    <strong>Text Classification Using Neural Network</strong><br>
    Use the movie review dataset for sentiment analysis<br>
</span>
<span style="font-family:verdana; font-size:1.4em;">
    <b>Following examples are included in the processing:</b><i>
    <ol>
        <li>Load dataset from csv file</li>
        <li>Preprocess data</li>
        <li>Create training and test dataset</li>
        <li>Tokenize the sequences</li>
        <li>Explore the tokenization</li>
        <li>Create a neural network</li>
        <li>Train model and make predictions</li>
        <li>Explore model performance</li>
    </ol> </i>   
</span>

</font>

In [ ]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import pandas as pd
import numpy as np
import timeit, time

from tensorflow.keras import layers
#from tensorflow.keras import losses
from tensorflow.keras import preprocessing

In [ ]:
# 50,000 movie reviews labeled as "positive" or "negative"
df = pd.read_csv("../datasets/IMDB Dataset.csv")

In [ ]:
# Map the sentiment (target variable) to numbers
df["sentiment"] = df["sentiment"].map({"positive": 1, "negative":0})

In [ ]:
pd.set_option('display.max_colwidth', None)
df.head(1)

In [ ]:
df.shape

## Preprocess function
<span style="font-family:verdana; font-size:1.2em;">
    Preprocess the input review to do the following:
    <ol>
        <li>Convert to lower case</li>
        <li>Remove html tags if found</li>
        <li>Remove puncutations</li>
    </ol>
</span>    

In [ ]:
import re, string

def preprocessData(input_data):
    lowercase = input_data.lower()
    stripped_html = lowercase.replace('<br />', ' ')
    retval = re.sub(r'[^\w\s]','', stripped_html)
    return retval

In [ ]:
# preprocess the review column
df['review'] = df['review'].map(preprocessData)

In [ ]:
pd.set_option('display.max_colwidth', None)
df.head()

## Create training and test datasets

In [ ]:
X_train = df["review"].values[:25000]
X_test = df["review"].values[25000:]

y_train = df["sentiment"].values[:25000]
y_test  = df["sentiment"].values[25000:]

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
X_train[0]

In [ ]:
y_train[0]

## Tokenize

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# define hyper parameters that can be modified
vocab_size = 10000
embedding_dim = 16
# If a sentence is shorter than max_length it will be padded, longer sentences will be truncated
max_length = 250  

In [ ]:
# Use Out of Vocabulary token rather than throwing away unknown words
tokenizer = Tokenizer(num_words=10000, oov_token = "<oov>")
tokenizer.fit_on_texts(X_train)
word_index = tokenizer.word_index

In [ ]:
# convert words to numbers and pad for the neural network to use as input
# if the sequence length is greater than max length then truncate it at the end
# if the sequence length is less than max length then pad it at the end
train_sequences = tokenizer.texts_to_sequences(X_train)
train_padded = pad_sequences(train_sequences, maxlen=max_length, 
                             padding = "post", truncating="post")


# tokenized using the word_index learned from the training data
testing_sequences = tokenizer.texts_to_sequences(X_test)
test_padded = pad_sequences(testing_sequences, maxlen=max_length, 
                            padding = "post", truncating="post")

In [ ]:
train_padded[1]

# Explore Tokenization

In [ ]:
# Reverses keys: keys become the values, and values become the keys so that 
# we can look a word up (display padded as ?)
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

# This is what will be fed in
print(decode_review(train_padded[1]))

In [ ]:
# This is the original text
print(X_train[1])

## Create Model
<span style="font-family:verdana; font-size:1.2em;">
    Model Parameters:
    <ol>
        <li><b>Embedding: </b>
            <ul>
                <li>Embedding layer stores one vector per word</li>
                <li>Converts sequences of word indices to sequences of vectors</li>
                <li>These vectors are trainable</li>
                <li>Once trained, words with similar meanings often have similar vectors</li>
                <li>This approach is more efficient than using a dense layer with one hot encoding</li>
            </ul>
        </li>
        <li><b>GlobalAveragePooling1D: </b>
            <ul>
                <li>Returns a fixed length output vector for each example by averaging over the sequence dimension</li>
                <li>Allows the model to handle input of variable length</li>
            </ul>
        </li>
        <li><b>Couple of Dense layers: </b>
            <ul>
                <li>Apply the dense layer with ReLU activation</li>
            </ul>
        </li>
    </ol>
    The last output layer use sigmoid to get probability of positive or negative sentiment
</span>

In [ ]:
# Create model
model = tf.keras.Sequential([
    # The Embedding layer is the key to text sentiment analysis
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(128, activation = 'relu'),
    tf.keras.layers.Dense(8, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [ ]:
# Compile model
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

In [ ]:
# Timer for measuring time taken to train the model
start = timeit.default_timer()

In [ ]:
history = model.fit(train_padded, y_train, epochs=5, validation_split = 0.2)

In [ ]:
# Stop timer and print training time
stop = timeit.default_timer()
execution_time = stop - start
exectime = time.strftime("%M:%S", time.gmtime(execution_time)) 
print("To train it took: {} mins".format(exectime))

## Explore Embeddings

In [ ]:
# Output from the Embedding layer
embeddings = model.layers[0]

weights = embeddings.get_weights()[0]
print(f"Vocabulary size: {weights.shape[0]},  Embedding dimensions: {weights.shape[1]}")

In [ ]:
# the shape is: 
# (the number of words in the corpus, the embedding dimensions)
weights.shape

In [ ]:
weights

In [ ]:
import io

# writing the vectors and their metadata out to file. 
# these 2 files ('vecs.tsv', 'meta.tsv') are used by the 
# TensorFlow projector (http://projector.tensorflow.org/)
# to plot/visualize the vectors/embeddings in 3D

# Output of the 16 values per word representation (embedding)
#      out_v are the weights (embedding)
#      out_m are the actual words associated with each embedding

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+ string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

In [ ]:
plot_graphs(history,'accuracy')

In [ ]:
plot_graphs(history, 'loss')

## Evaluate the model

In [ ]:
test_loss, test_acc = model.evaluate(test_padded, y_test)

print("Test accuracy: ", test_acc)

In [ ]:
# predict on a positive sample

sample_text_to_predict = ["The movie was great. The animation and the graphics was excellent. I would recommend this movie."]

train_sequences = tokenizer.texts_to_sequences(sample_text_to_predict)
pos_padded = pad_sequences(train_sequences, maxlen=max_length, padding = "post", truncating="post")

#  make prediction
prediction = model.predict(pos_padded)

print(prediction)

In [ ]:
# predict on a negative sample

sample_text_to_predict = ["The movie was horrible. The animation and the graphics were wrost. I would not recommend this movie."
]

train_sequences = tokenizer.texts_to_sequences(sample_text_to_predict)
neg_padded = pad_sequences(train_sequences, maxlen=max_length, padding = "post", truncating="post")

#  make prediction
prediction = model.predict(neg_padded)

print(prediction)